#### Silver Layer Processing - Worldwide Weather Event API 

In [94]:
from pyspark.sql.functions import col, explode, arrays_zip, to_date
from pyspark.sql.types import TimestampType
import requests
url = f"https://us1.api-bdc.net/data/reverse-geocode-client?latitude={latitude}&longitude={longitude}&localityLanguage=en"
response = requests.get(url)
city = response.json()["city"]

StatementMeta(, 990f740a-b087-46fc-bc0b-ed9f6207b077, 96, Finished, Available, Finished)

StatementMeta(, 990f740a-b087-46fc-bc0b-ed9f6207b077, 102, Finished, Available, Finished)

In [101]:
# JSON-Datei einlesen
df = spark.read.option("multiline", "true").json(f"Files/{city}_weather.json")

StatementMeta(, 990f740a-b087-46fc-bc0b-ed9f6207b077, 103, Finished, Available, Finished)

In [99]:
df_base = df.select(
    col("elevation").cast("double").alias("elevation"),
    col("generationtime_ms").cast("double").alias("generationtime_ms"),
    col("latitude").cast("double").alias("latitude"),
    col("longitude").cast("double").alias("longitude"),
    col("timezone").cast("string").alias("timezone"),
    col("timezone_abbreviation").cast("string").alias("timezone_abbreviation"),
    col("utc_offset_seconds").cast("bigint").alias("utc_offset_seconds"),

    col("hourly.temperature_2m").alias("temperature"),
    col("hourly.time").alias("time"),

    col("hourly_units.temperature_2m").alias("temperature_unit"),
    col("hourly_units.time").alias("time_format")
)

df_flat = df_base.select(
    "elevation",
    "generationtime_ms",
    "latitude",
    "longitude",
    "timezone",
    "timezone_abbreviation",
    "utc_offset_seconds",
    "temperature_unit",
    "time_format",
    explode(arrays_zip("time", "temperature")).alias("hourly_data") 
).select(
    "elevation",
    "generationtime_ms",
    "latitude",
    "longitude",
    "timezone",
    "timezone_abbreviation",
    "utc_offset_seconds",
    "temperature_unit",
    "time_format",
    col("hourly_data.time").cast(TimestampType()).alias("time"),
    col("hourly_data.temperature").cast("double").alias("temperature")
)

# Speichere die Tabelle
df_flat.write.mode("append").saveAsTable("weather_data_silver")


StatementMeta(, 990f740a-b087-46fc-bc0b-ed9f6207b077, 101, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 887a16b8-e988-4c8a-b642-55571dd7951a)